# PART ONE

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
soup = BeautifulSoup(requests.get(url).text,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
rows = table.find_all('tr')
t=[]
for tr in rows:
    td = tr.find_all('td')
    row = [tr.text.rstrip('\n') for tr in td]
    t.append(row)
df=pd.DataFrame(t)
df.columns=['Postcode','Borough','Neighbourhood']
df.drop([0],axis=0,inplace=True)
df.reset_index()
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df1=df.groupby("Postcode").agg(lambda x:','.join(set(x)))
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']
df1.shape
df1

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
M1E,Scarborough,"Morningside,West Hill,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Scarborough Village West,Cliffcrest,Cliffside"


In [15]:
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")
df1['Latitude']=geo_data['Latitude'].values
df1['Longitude']=geo_data['Longitude'].values
df1.head()

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## MAP

In [14]:
import folium
center_lat=df1.Latitude.mean()
center_long=df1.Longitude.mean()
venues_map = folium.Map(location=[center_lat, center_long], zoom_start=13)
for lat, lng, label in zip(df1.Latitude, df1.Longitude, df1.categories):
    folium.features.CircleMarker(
    [df1[5]['Latitude'], df1[5]['Longitude']],
    radius=2,
    color='red',
    popup=r['Neighbourhood'],
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
    ).add_to(venues_map)


AttributeError: 'DataFrame' object has no attribute 'categories'

In [19]:
import folium
center_lat=df1.Latitude.mean()
center_long=df1.Longitude.mean()
venues_map = folium.Map(location=[center_lat, center_long], zoom_start=13)
data = df1[df1['Borough'] == 'Toronto'].reset_index(drop=True)
for lat, lng, label in zip(df1['Latitude'], df1['Longitude'],df1['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map)  
    
venues_map